In [31]:
#this program will combine images to make a master frame

#if you use this code, please cite Oelkers & Stassun 2018

#import the relevant libraries for basic tools
#import pyfits
import numpy as np
import scipy
from scipy import stats
from os import path
import math
import time
import csv
import astropy
#import pyearth

#import relevant libraries for a list
import glob, os
from os import listdir
from os.path import isfile, join, exists
#from pyfits import getheader #new libraries added
from astropy.io import fits
from astropy.wcs import wcs
#from astropy.io.fits import getval
#from pyfits import getval
#from astropy.nddata.utils import Cutout2D

In [32]:
###UPDATE HERE#####
#what field are you looking at?
camera = '2'
ccd = '2'
blknum = 50 #how many images go into each holder?

#this program will select the images to make the E and W master frames
#useful directories
cdedir = '/home/thaismadruga/Documentos/FFIs/' #onde está o código 
clndir = '/home/thaismadruga/Documentos/FFIs/' #onde as imagens reduzidas ficam
rawdir = '/home/thaismadruga/Documentos/FFIs/' #onde as imagens 'cruas' ficariam 

###END UPDATE###

#get the image list and the number of files which need reduction
#files = [f for f in glob.glob("*.fits") if isfile(join(rawdir, f))]
files = np.loadtxt(cdedir+'images.dat',dtype = 'str' ,unpack = 1,usecols = (0)) 
files.sort()
print (files)
nfiles = len(files) #devolve o comprimento de files
ref, rhead = fits.getdata(rawdir+files[0], header = True, ignoring_missing_end=True)

#iterate through the files
cnt = 0 #contador para o numero de imagens usadas
kk = 0 #the jumper for file placement
align = 1

for ii in range(0,nfiles):
   #np.loadtxt(cdedir+'images.dat', dtype = 'string', unpack = 1, usecols = 0)
#for ii in range(0, nfiles):
    hdu = files[ii].split('.') #gives an array like ['kplr2009114174833_ffi-cal', 'fits'
    hdul = fits.open('images.dat')
    #print (hdul[0].header['NAXIS2'])
    
    
    #update the name to be appropriate for what was done to the file
    if (align == 1):
        finnme = hdu[0]+'_sa.fits'
        #st = time.time()
        #sts = time.strftime("%c")
        #read in the image
        hdulist = fits.open(rawdir+files[ii])
        
        for j in range(1,len(hdulist)):#counting on that there is an primary header (the zero in the list)
            orgimg, header = fits.getdata(rawdir+files[ii], header = True, ignoring_missing_end=True)
            print (header) #acrescentei
            #print(np.shape(orgimg))
            w = wcs.WCS(header)
                    
    

    if (ii == 1):  # get size on first iteration only
        #print clndir+files[0]
        nx = fits.open(clndir+files[0],'NAXIS2')
        ny = fits.open(clndir+files[0], 'NAXIS1')
        #nx = '1070'
        #ny = '1132'
        #nx = pyfits.getval(clndir+files[0],'NAXIS2') 
        #ny = pyfits.getval(clndir+files[0],'NAXIS1')
        #aa = pyfits.getheader(clndir+files[ii], 1)
        #nx = aa['NAXIS2']
        #print nx
        #all_data = np.ndarray(shape=(blknum,nx,ny))
        #expt = np.zeros(blknum) 

        #read in the image
        img_data = pyfits.getdata(clndir+files[ii])
        expt[cnt] = pyfits.getval(clndir+files[ii],'EXPOSURE')

        #add the image to the vector
        all_data[cnt] = img_data 
        cnt = cnt+1

        if (ii % 10 == 0) and (ii > 0):
                print ('Finished with 10 images at '+str(time.strftime("%a %d %b %Y %H:%M:%S"))+'.') 
                

        if (ii == nfiles-1) or ((ii+1) % blknum == 0):
                
                #median combine the data
                combined_data = np.median(all_data,axis=0)

                # Write data to new file    
                new_image = pyfits.PrimaryHDU(combined_data)
                new_image.header.set('NUMCOMB', cnt)
                new_image.header.set('EXPTIME', np.median(expt))

                #print the file with the appropriate counter
                if (kk < 10):
                        new_image.writeto(cdedir+'frames/'+camera+'_'+ccd+'_master_0'+str(kk)+'.fits',clobber=True)
                else: 
                        if (kk >= 10) and (kk < 100):
                                new_image.writeto(cdedir+'frames/'+camera+'_'+ccd+'_master_'+str(kk)+'.fits',clobber=True)

                print ("The master frame hold was created using a median of "+str(cnt)+" images.")
                kk = kk+1
                cnt = 0

#clear the data file
all_data = np.ndarray(shape=(blknum,nx,ny))
expt = np.zeros(blknum)

del all_data, img_data # clear up some memory

['kplr2009114174833_ffi-cal.fits' 'kplr2009114204835_ffi-cal.fits'
 'kplr2009115002613_ffi-cal.fits' 'kplr2009115053616_ffi-cal.fits'
 'kplr2009115080620_ffi-cal.fits' 'kplr2009115131122_ffi-cal.fits'
 'kplr2009115173611_ffi-cal.fits' 'kplr2009116035924_ffi-cal.fits']


OSError: Header missing END card.